In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from my_functions import *
from scipy.stats import median_abs_deviation
from three_filter import three_filter_method
import seaborn as sns

In [ ]:
## Load everything
pm_data = np.load('npy/pm_eLAE_15deg_uniform.npy')
pm_data_noerr = np.load('npy/pm_LAE_15deg_uniform.npy')
f_name = 'Source_cats/Source_cat_LAE_15deg_uniform.npy'
mock = np.load(f_name, allow_pickle=True).item()

errors = np.load('npy/median_errors.npy')

pm_err = (np.ones(pm_data.shape).T * errors).T

filters_tags = load_filter_tags()
tcurves = load_tcurves(filters_tags)
w_central = np.array(central_wavelength(tcurves))

bb_fwhm = [
    nb_fwhm(tcurves, -4, True),
    nb_fwhm(tcurves, -3, True),
    nb_fwhm(tcurves, -2, True),
    nb_fwhm(tcurves, -1, True)
]

In [ ]:
nb_c = 13

NB = pm_data[nb_c]
BB_LC = pm_data[-3]
BB_LU = pm_data[-2]
NB_err = pm_err[nb_c]
BB_LC_err = pm_err[-3]
BB_LU_err = pm_err[-2]
t_NB = np.array(tcurves['t'][nb_c])
t_BB_LC = np.array(tcurves['t'][-3])
t_BB_LU = np.array(tcurves['t'][-2])
w_NB = np.array(tcurves['w'][nb_c])
w_BB_LC = np.array(tcurves['w'][-3])
w_BB_LU = np.array(tcurves['w'][-2])
w_EL = w_central[nb_c]

F_EL, A, B, A_err, B_err = three_filter_method(
    NB, BB_LC, BB_LU, NB_err, BB_LC_err, BB_LU_err,
    t_NB, w_NB, t_BB_LC, t_BB_LU, w_BB_LC, w_BB_LU, w_EL
)

f_cont = A * w_EL + B
f_cont_var = w_EL**2 * A_err**2 + B_err**2
EW0 = F_EL / f_cont * 1215.67/w_EL

line = (
    (pm_data[nb_c] - f_cont > 3*(pm_err[nb_c]**2 + f_cont_var) ** 0.5)
    & (EW0 > 20)
)

fig, ax = plt.subplots(figsize=(12, 9))

ax.plot(EW0[line], mock['EW_Arr'][line], marker='.', linestyle='', c='C1')

ax.plot(
    np.linspace(0, 300, 300), np.linspace(0, 300, 300),
    linestyle='--', c='red', linewidth=3
)

ax.text(
    20, 100, 'bias = {0:0.2f} \nsigma = {1:0.2f}'.format(
        np.median(
            EW0[line] - mock['EW_Arr'][line],
            ),
        median_abs_deviation(
            EW0[line] - mock['EW_Arr'][line],
            ),
        ),
    fontsize=15
)

ax.set_ylim((0, 140))
ax.set_xlim((0, 140))

plt.show()

EW0_line_diff = EW0[line] - mock['EW_Arr'][line]

line_3f = line

print(len(np.where(line)[0]))

In [ ]:
# EW diff
w_lya = 1215.67

for N_nb in range(4, 6):
    fig, ax = plt.subplots(figsize=(12, 9))

    EW_ret_Arr = np.ones(mock['EW_Arr'].shape) * -999.
    fwhm = nb_fwhm(tcurves, nb_c, True)
    cont_est, cont_err = stack_estimation(
        pm_data, pm_err, nb_c, N_nb, w_central
    )

    EW_ret = (pm_data[nb_c] - cont_est) * fwhm / cont_est\
        / (w_central[nb_c] / w_lya)
    line = (
        (pm_data[nb_c] - cont_est > 3 * (pm_err[nb_c]**2 + cont_err**2)**0.5)
        & (EW_ret > 20)
    )
    print(len(np.where(line)[0]))

    EW_ret_Arr[line] = EW_ret[line]

    mask999 = np.where(EW_ret_Arr != -999.)

    ax.plot(
        mock['EW_Arr'][mask999], EW_ret_Arr[mask999],
        marker='.', markersize=12, linestyle='', alpha=0.5,
        label='Stack'
        )

    ax.plot(
        np.linspace(0, 300, 300), np.linspace(0, 300, 300),
        linestyle='--', c='red', linewidth=3
        )

    # ax.text(
    #     20, 100, 'bias = {0:0.2f} \nsigma = {1:0.2f}'.format(
    #         np.median(
    #             EW_ret_Arr[mask999] - mock['EW_Arr'][mask999],
    #             ),
    #         median_abs_deviation(
    #             EW_ret_Arr[mask999] - mock['EW_Arr'][mask999],
    #             ),
    #         ),
    #     fontsize=15
    # )

    ax.plot(
        EW0[line], mock['EW_Arr'][line],
        marker='.', linestyle='', c='C1', markersize=12, alpha=0.5,
        label='3-filter'
        )

    ax.set_ylim((0, 200))
    ax.set_xlim((0, 200))
    ax.set_xlabel('Real EW$_0$ ($\AA$)', fontsize=15)
    ax.set_ylabel('Retrieved EW$_0$ ($\AA$)', fontsize=15)
    ax.set_title('N_nb = {}'.format(N_nb), fontsize=20)
    ax.legend(fontsize=15)
    plt.show()

line_st = line
EW0_stack_diff = EW_ret_Arr[mask999] - mock['EW_Arr'][mask999]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

histbins = np.linspace(-75, 75, 30)
ax.hist(EW0_stack_diff, bins=histbins, alpha=0.6, label='Stack')
ax.hist(EW0_line_diff, bins=histbins, alpha=0.6, label='3-filter')

ax.set_xlabel('$\Delta$EW$_0$ ($\AA$)', fontsize=15)
ax.set_ylabel('N', fontsize=15)

ax.legend(fontsize=15)

plt.show()
print(
    '3-filter: mean = {0:0.2f}, std = {1:0.2f}, N_Sources = {2}({3:0.1f}%)'.format(
    np.average(EW0_line_diff), np.std(EW0_line_diff), 
    len(EW0_line_diff), len(EW0_line_diff)/pm_data.shape[1]*100
    )
)
print(
    'Stack   : mean = {0:0.2f}, std = {1:0.2f}, N_Sources = {2}({3:0.1f}%)'.format(
    np.average(EW0_stack_diff), np.std(EW0_stack_diff), 
    len(EW0_stack_diff), len(EW0_stack_diff)/pm_data.shape[1]*100
    )
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

histbins = np.linspace(10, 150, 10)
ax.hist(mock['EW_Arr'], bins=histbins, histtype='step', label='All')
ax.hist(mock['EW_Arr'][line_st], bins=histbins, label='Stack selection')
ax.hist(mock['EW_Arr'][line_3f], bins=histbins, label='3-filter selection')

ax.set_xlabel('Real EW$_0$ ($\AA$)', fontsize=15)
ax.set_ylabel('N', fontsize=15)

ax.legend(fontsize=14)
plt.show()

In [ ]:
def conf_matrix(line_Arr, z_Arr, nb_c):
    w_in = list(nb_fwhm(tcurves, nb_c))
    w_in.sort()
    w_in += np.array([-5, 5])
    z_in = np.array([w / 1215.67 - 1 for w in w_in])
    
    TP = len(np.where( line_Arr &  ((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])
    FP = len(np.where( line_Arr & ~((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])
    TN = len(np.where(~line_Arr & ~((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])
    FN = len(np.where(~line_Arr &  ((z_in[0] < z_Arr) & (z_Arr < z_in[1])))[0])

    return np.array([[TP, FP], [FN, TN]])

In [ ]:
cm = np.zeros((2, 2))
n = 0
N_nb = 6
ew0min = 20
for nb_c in range(13,14):
    z_nb = w_central[nb_c] / 1215.67 - 1

    pm_err = (np.ones(pm_data.shape).T * errors).T
    cont_est, cont_err = stack_estimation(
        pm_data, pm_err, nb_c, N_nb, w_central
        )

    diff_err = (errors[nb_c]**2 + cont_err**2)**0.5
    fwhm = nb_fwhm(tcurves, nb_c, True)

    line = (
    (pm_data[nb_c] - cont_est > 3*diff_err)
    & (pm_data[nb_c] - cont_est > ew0min * (1 + z_nb) * cont_est / fwhm)
    )

    cm += conf_matrix(line, mock['redshift_Lya_Arr'], nb_c)

fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, ax=ax, cbar=False)
ax.set_xlabel('REAL', fontsize=15)
ax.set_ylabel('SELECTED', fontsize=15)
ax.set_title('Stack', fontsize=20)
ax.set_xticklabels(['True', 'False'])
ax.set_yticklabels(['True', 'False'])
plt.show()

print('True Positive Rate: {0:0.1f}%'.format(100 * cm[0,0] / (cm[0,0] + cm[1,0])))
print('Accuracy: {0:0.1f}%'.format(100 * (cm[0,0] + cm[1,1]) / (np.sum(cm))))

In [ ]:
cm = np.zeros((2, 2))
n = 0
N_nb = 5
ew0min = 20
for nb_c in range(13, 14):
    NB = pm_data[nb_c]
    BB_LC = pm_data[-3]
    BB_LU = pm_data[-2]
    NB_err = pm_err[nb_c]
    BB_LC_err = pm_err[-3]
    BB_LU_err = pm_err[-2]
    t_NB = np.array(tcurves['t'][nb_c])
    t_BB_LC = np.array(tcurves['t'][-3])
    t_BB_LU = np.array(tcurves['t'][-2])
    w_NB = np.array(tcurves['w'][nb_c])
    w_BB_LC = np.array(tcurves['w'][-3])
    w_BB_LU = np.array(tcurves['w'][-2])
    w_EL = w_central[nb_c]

    F_EL, A, B, A_err, B_err = three_filter_method(
        NB, BB_LC, BB_LU, NB_err, BB_LC_err, BB_LU_err,
        t_NB, w_NB, t_BB_LC, t_BB_LU, w_BB_LC, w_BB_LU, w_EL
    )

    f_cont = A * w_EL + B
    f_cont_var = w_EL**2 * A_err**2 + B_err**2
    EW0 = F_EL / f_cont * 1215.67/w_EL

    line = (
        (pm_data[nb_c] - f_cont > 3*(pm_err[nb_c]**2 + f_cont_var) ** 0.5)
        & (EW0 > 10)
    )

    cm += conf_matrix(line, mock['redshift_Lya_Arr'], nb_c)

fig, ax = plt.subplots()
sns.heatmap(cm, annot=True, ax=ax, cbar=False)
ax.set_xlabel('REAL', fontsize=15)
ax.set_ylabel('SELECTED', fontsize=15)
ax.set_title('3-filter', fontsize=20)
ax.set_xticklabels(['True', 'False'])
ax.set_yticklabels(['True', 'False'])
plt.show()

print('True Positive Rate: {0:0.1f}%'.format(100 * cm[0,0] / (cm[0,0] + cm[1,0])))
print('Accuracy: {0:0.1f}%'.format(100 * (cm[0,0] + cm[1,1]) / (np.sum(cm))))